# Data

In [1]:
# R PACKAGES
if(!require("pacman")) install.packages("pacman")
pacman::p_load(arrow, tidyverse, janitor, readxl, stringdist)

Loading required package: pacman


We used three ICD code lists to identify MSK conditions:
* All of the ICD10 M codes from the 2019 [CMS](https://www.cms.gov/medicare/coding-billing/icd-10-codes/icd-10-cm-icd-10-pcs-gem-archive#:~:text=2019%20ICD%2D10%20CM%20%26%20PCS%20files) ICD-10 code list
* A list of codes ('power') used to identify MSK conditions in Canada from [Power et al](https://pubmed.ncbi.nlm.nih.gov/35365584/). 
* A list of codes ('hwa') used to identify “less-urgent” MSK conditions in Australia [Thompson et al](https://ro.uow.edu.au/ahsri/375/). 

The power codes are ICD-10CA codes in short (3-4 character) format. To get these codes to match the other code lists, they were mapped into ICD-10 WHO codes. the WHO ICD-10 codes were extracted from the Python package [simple_icd_10](https://github.com/StefanoTrv/simple_icd_10/releases), which is the extedned code list from th WHO [page](https://icd.who.int/browse10/2019/en#) for the 2019 codes and from there mapped onto the 2019 CMS ICD-10CM codes

The hwa codes were in ICD-10-AM format. Unlike ICD-10-CA, Australia actually provide [maps](https://www.ihacpa.gov.au/resources/icd-10-am-and-achi-mapping-tables) (11th edition) to ICD-10-WHO, and these WHO codes were mapped into the CMS codes


In [2]:
who_2019 <- read_delim(
  "icd102019syst_codes.txt", 
  delim = ";", 
  col_names=F, 
  show_col_types=F
) |> 
  select(
    who_code = X8, 
    who_desc = X9
  )

#CMS ICD10 CM code list
cms <- read_lines(
  "icd10cm_order_2019.txt") |> 
  as_tibble() |> 
  separate_wider_regex(
    value,
    patterns = c(
    id = "[^\\s]+",         # First non-whitespace block
    "\\s+",                 # One or more spaces
    rest = ".*"             # Everything else
      )
  ) |> 
  separate_wider_regex(
    rest,
    patterns = c(
      cms_code = "[^\\s]+",         # First non-whitespace block
      "\\s+",                 # One or more spaces
      rest_again = ".*"             # Everything else
    )
  ) |> 
  separate_wider_regex(
    rest_again,
    patterns = c(
      level = "[^\\s]+",         # First non-whitespace block
      "\\s+",                 # One or more spaces
      rest_desc = ".*"             # Everything else
    )
  ) |> 
  mutate(
    rest_desc = str_replace_all(rest_desc, "\\s{2,}", "!!!")
  ) |> 
    separate_wider_delim(
      rest_desc,
      delim = "!!!",
      too_few = "align_start",
      too_many = 'merge',
      names = c("cms_desc", "cms_desc_long")
    ) |> 
  select(cms_code, cms_desc) |> 
  distinct(cms_code, .keep_all = T)

Warning message:
One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat) 


In [3]:
# HWA (thompson) codes
# getting the codes from the text extracted from the pdf to a tibble is  long and is shown in the hwa.r file
source("hwa.R")
hwa<- hwa_f() |> 
  mutate(
    code_length = str_count(hwa_code),
    code_letter = str_sub(hwa_code,1,1)
  ) |> 
  distinct(hwa_code, .keep_all = T) 
  # see whic codes have children/parents
  hwa <- hwa |> 
    mutate(
      has_child = map_lgl(hwa_code, function(code) {
      any(str_starts(hwa$hwa_code, code) & nchar(hwa$hwa_code) > nchar(code))
    }) ,
    is_child = map_lgl(hwa_code, function(code) {
      # Generate all prefixes of the code excluding the full code itself
      prefixes <- str_sub(code, 1, seq_len(str_length(code) - 1))
      # Check if any prefix exists in the list of codes
      any(prefixes %in% hwa$hwa_code)
    })
  ) |> 
    # I dont need the Z codes, these are not diagnoses
  filter(
    code_letter != "Z"
  )

write_csv (hwa, "hwa_codes.csv")

New names:
• `` -> `...1`


# Functions

In [4]:
describe_function <- function(df, col_name){

  print("N codes in list")
  df |> 
    count() |> 
    print()

  cat("\nCode lengths in list")
  df |> 
    group_by(code_length) |> 
    count() |> 
    print()

  cat("\nCode letters in list")
  df |> 
    group_by(code_letter) |>
    count() |>
    mutate(
      `%` = round(n/nrow(df)*100,1)
    ) |>
    print(n=120)

  cat("\nCode lengths and letters in list")
  df |> 
    group_by(code_length, code_letter) |>
    count() |>
    mutate(
      `%` = round(n/nrow(df)*100,1)
    ) |>
    ungroup() |> 
    arrange(code_letter, code_length) |> 
    print(n=120)
}

# Describe HWA code list

In [5]:
describe_function(hwa, hwa_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1   496

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3     6
2           4   341
3           5   149

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0.2
2 G               5   1  
3 I               2   0.4
4 M             150  30.2
5 Q               1   0.2
6 R              13   2.6
7 S             297  59.9
8 T              27   5.4

Code lengths and letters in list# A tibble: 15 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0.2
 2           4 G               5   1  
 3           4 I               2   0.4
 4           3 M               1   0.2
 5           4 M              80  16.1
 6           5 M              69  13.9
 7           4 Q               1   0.2
 8           3 R               1   0.2


## Parent and child codes

#### Describe the parent codes

In [6]:
describe_function(hwa |> 
  filter(has_child), hwa_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1    56

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           3     2
2           4    54

Code letters in list# A tibble: 2 × 3
# Groups:   code_letter [2]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M              17  30.4
2 S              39  69.6

Code lengths and letters in list# A tibble: 4 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           3 M               1   1.8
2           4 M              16  28.6
3           3 S               1   1.8
4           4 S              38  67.9


In [7]:
hwa |> 
  filter(has_child) |>
  print(n=120)

# A tibble: 56 × 6
   hwa_code hwa_desc                                code_length code_letter has_child is_child
   <chr>    <chr>                                         <int> <chr>       <lgl>     <lgl>   
 1 M069     Rheumatoid arthritis unspecified                  4 M           TRUE      FALSE   
 2 M10      Gout                                              3 M           TRUE      FALSE   
 3 M109     Gout unspecified                                  4 M           TRUE      TRUE    
 4 M139     Arthritis unspecified                             4 M           TRUE      FALSE   
 5 S508     Other superficial injuries of forearm             4 S           TRUE      FALSE   
 6 S520     Fracture of upper end of ulna                     4 S           TRUE      FALSE   
 7 S521     Fracture of upper end of radius                   4 S           TRUE      FALSE   
 8 S522     Fracture of shaft of ulna                         4 S           TRUE      FALSE   
 9 S523     Fracture of shaft o

see which codes are children

In [8]:
hwa |> 
  filter(is_child) |>
  print(n=120)

# A tibble: 106 × 6
    hwa_code hwa_desc                                 code_length code_letter has_child is_child
    <chr>    <chr>                                          <int> <chr>       <lgl>     <lgl>   
  1 S4300    Dislocation of shoulder unspecified                5 S           FALSE     TRUE    
  2 S4301    Anterior dislocation of humerus                    5 S           FALSE     TRUE    
  3 S431     Dislocation of acromioclavicular joint             4 S           FALSE     TRUE    
  4 S433     Disloc oth unsp parts shoulder girdle              4 S           FALSE     TRUE    
  5 S434     Sprain and strain of shoulder joint                4 S           FALSE     TRUE    
  6 S435     Sprain strain acromioclavicular joint              4 S           FALSE     TRUE    
  7 S436     Sprain strain sternoclavicular joint               4 S           FALSE     TRUE    
  8 S437     Sprain strain oth unsp shoulder girdle             4 S           FALSE     TRUE    
  9 M0699 

See which codes have 1-2-1 mapping in the CMS code list

In [9]:
hwa <- hwa |> 
  mutate(
    has_cms = map_lgl(hwa_code,  ~ .x %in% cms$cms_code)
  )

In [10]:
print("N codes with 1-2-1 mapping")
hwa |>
  filter(has_cms) |>
  count() |>
  mutate(
    `%` = n/nrow(hwa)*100
  ) |>
  print()

[1] "N codes with 1-2-1 mapping"
# A tibble: 1 × 2
      n   `%`
  <int> <dbl>
1   387  78.0


# Mapping

## Parent Codes

I will only need to map the parent codes and the single codes.  
First, map the parent codes

In [11]:
print("parents with 1-2-1 mapping")
hwa |>
  filter(has_cms) |>
  filter(has_child) |>
  count() |>
  mutate(
    `%` = n/nrow(hwa |> filter(has_child))*100
  ) |>
  print()


[1] "parents with 1-2-1 mapping"
# A tibble: 1 × 2
      n   `%`
  <int> <dbl>
1    54  96.4


Most parent codes have 1-2-1 maps. Map these and check the mapping

In [12]:
print("which parents have 1-2-1 mapping?")
parent_cms <- hwa |>
  filter(has_cms) |>
  filter(has_child) |>
  crossing(cms) |>
  filter(
    hwa_code == cms_code
  )
options(width=200)
parent_cms |> 
  select(hwa_code, hwa_desc, cms_desc, cms_code) |>
  print(n=120)

[1] "which parents have 1-2-1 mapping?"
# A tibble: 54 × 4
   hwa_code hwa_desc                                cms_desc                                                                                                                       cms_code
   <chr>    <chr>                                   <chr>                                                                                                                          <chr>   
 1 M069     Rheumatoid arthritis unspecified        Rheumatoid arthritis, unspecified                                                                                              M069    
 2 M10      Gout                                    Gout                                                                                                                           M10     
 3 M109     Gout unspecified                        Gout, unspecified                                                                                                              M109    
 

In [13]:
describe_function(parent_cms, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1    54

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           3     2
2           4    52

Code letters in list# A tibble: 2 × 3
# Groups:   code_letter [2]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M              15  27.8
2 S              39  72.2

Code lengths and letters in list# A tibble: 4 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           3 M               1   1.9
2           4 M              14  25.9
3           3 S               1   1.9
4           4 S              38  70.4


These look fine, the mappings look correct, and the sub-codes for these can be extracted becuase they are all going to be MSK (accoring to the HWA definition)

In [14]:
parent_cms_map <- parent_cms |>
  select(-cms_desc, -cms_code) |> 
  crossing(cms) |> 
  filter(
    startsWith(cms_code, hwa_code)
  )  

In [15]:
describe_function(parent_cms_map, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1 11711

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           3   638
2           4 11073

Code letters in list# A tibble: 2 × 3
# Groups:   code_letter [2]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M             426   3.6
2 S           11285  96.4

Code lengths and letters in list# A tibble: 4 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           3 M             162   1.4
2           4 M             264   2.3
3           3 S             476   4.1
4           4 S           10809  92.3


Now check out the parent codes that do not have 1-2-1 mapping

In [16]:
hwa |>
  filter(!has_cms,has_child) |>
  print(n=120)

# A tibble: 2 × 7
  hwa_code hwa_desc                         code_length code_letter has_child is_child has_cms
  <chr>    <chr>                                  <int> <chr>       <lgl>     <lgl>    <lgl>  
1 M139     Arthritis unspecified                      4 M           TRUE      FALSE    FALSE  
2 M665     Spont rupture unspecified tendon           4 M           TRUE      FALSE    FALSE  


I use the [UMLS](https://uts.nlm.nih.gov/uts/umls/home) browser to find realted terms for these codes using the code description   
For M139 Arthritis unspecified the realted term is "Arthritis", which the UMLS maps down to ICD-10-CM M19.90 Arthritis Not Otherwise Specified.   
for M665 Spontaneous rupture unspecified tendon, the realted term is Non-traumatic tendon rupture, which UMLS maps to ICD-10-CM M66.9 Spontaneous rupture of unspecified tendon.   
So for these two codes, they can be mapped to the related terms

In [17]:
parent_cms_umls <- hwa |> 
  filter(!has_cms, has_child) |>
  mutate(
    code_to_map = case_when(
      hwa_code == "M139" ~ "M1990",
      hwa_code == "M665" ~ "M669"
    )
  ) |>
  crossing(cms) |>
  filter(
    startsWith(cms_code, code_to_map)
  ) |> 
  select(-code_to_map) 

parent_cms_join <- bind_rows(parent_cms_map, parent_cms_umls)

The child codes will not need to be mapped, as they've been mapped via their parent code

### Describe the parent/child code list

In [18]:
describe_function(parent_cms_join, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1 11713

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           3   638
2           4 11075

Code letters in list# A tibble: 2 × 3
# Groups:   code_letter [2]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M             428   3.7
2 S           11285  96.3

Code lengths and letters in list# A tibble: 4 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           3 M             162   1.4
2           4 M             266   2.3
3           3 S             476   4.1
4           4 S           10809  92.3


## Single codes

### Describe the single codes

In [19]:
single_codes <- hwa |> 
  filter(!has_child & !is_child) 

describe_function(single_codes, hwa_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1   336

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3     4
2           4   281
3           5    51

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0.3
2 G               5   1.5
3 I               2   0.6
4 M             102  30.4
5 Q               1   0.3
6 R              13   3.9
7 S             185  55.1
8 T              27   8  

Code lengths and letters in list# A tibble: 14 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0.3
 2           4 G               5   1.5
 3           4 I               2   0.6
 4           4 M              64  19  
 5           5 M              38  11.3
 6           4 Q               1   0.3
 7           3 R               1   0.3
 8           4 R              12   3.6


Now map the single codes and keep those with 1-2-1 mapping. Print these and manually check the maps

In [20]:
single_cms <- single_codes |> 
  filter(has_cms) |> 
  crossing(cms) |>
  filter(
    hwa_code == cms_code
  ) 
options(width=200)
single_cms |> 
  select(hwa_code, hwa_desc, cms_desc, cms_code) |>
  print(n=400)

# A tibble: 253 × 4
    hwa_code hwa_desc                                 cms_desc                                                                                                                                  cms_code
    <chr>    <chr>                                    <chr>                                                                                                                                     <chr>   
  1 D480     Neoplm unc unk beh bone articular cart   Neoplasm of uncertain behavior of bone/artic cartl                                                                                        D480    
  2 G439     Migraine unspecified                     Migraine, unspecified                                                                                                                     G439    
  3 G442     Tension type headache                    Tension-type headache                                                                                                     

Describe the single codes with 1-2-1 maps

In [21]:
describe_function(single_cms, hwa_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1   253

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3     4
2           4   223
3           5    26

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0.4
2 G               5   2  
3 I               2   0.8
4 M              81  32  
5 Q               1   0.4
6 R               9   3.6
7 S             152  60.1
8 T               2   0.8

Code lengths and letters in list# A tibble: 13 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0.4
 2           4 G               5   2  
 3           4 I               2   0.8
 4           4 M              61  24.1
 5           5 M              20   7.9
 6           4 Q               1   0.4
 7           3 R               1   0.4
 8           4 R               8   3.2


All of the single codes with 1-2-1 maps look OK, except T07. In ICD-10-CM, T07 has sub codes for Suicide attempt, and thus should not be included. This code will be removed. All other can be safely mapped, and thier sub-codes can be safely extracted. Map these and describe the dataset

In [22]:
single_cms_mapped <- single_cms |> 
  filter(hwa_code != "T07") |> 
  select(-cms_desc, -cms_code) |>
  crossing(cms) |>
  filter(
    startsWith(cms_code, hwa_code)
  ) |> 
  # I dont need the codes I have already mapped using the parent codes
  anti_join(
    parent_cms_join,
    join_by(cms_code)
  )  
  
describe_function(single_cms_mapped, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1  9214

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3    31
2           4  8914
3           5   269

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0  
2 G              32   0.3
3 I              27   0.3
4 M             653   7.1
5 Q               1   0  
6 R              11   0.1
7 S            8485  92.1
8 T               4   0  

Code lengths and letters in list# A tibble: 12 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0  
 2           4 G              32   0.3
 3           4 I              27   0.3
 4           4 M             583   6.3
 5           5 M              70   0.8
 6           4 Q               1   0  
 7           3 R               1   0  
 8           4 R              10   0.1


In [23]:
cms_join <- bind_rows(parent_cms_join, single_cms_mapped)

In [24]:
describe_function(cms_join, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1 20927

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3   669
2           4 19989
3           5   269

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0  
2 G              32   0.2
3 I              27   0.1
4 M            1081   5.2
5 Q               1   0  
6 R              11   0.1
7 S           19770  94.5
8 T               4   0  

Code lengths and letters in list# A tibble: 13 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0  
 2           4 G              32   0.2
 3           4 I              27   0.1
 4           3 M             162   0.8
 5           4 M             849   4.1
 6           5 M              70   0.3
 7           4 Q               1   0  
 8           3 R               1   0  


Now examine the single codes without 1-2-1 mapping

In [25]:
single_nocms <- hwa |> 
  filter(!has_cms, !has_child & !is_child)

describe_function(single_nocms, hwa_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1    83

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           4    58
2           5    25

Code letters in list# A tibble: 4 × 3
# Groups:   code_letter [4]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M              21  25.3
2 R               4   4.8
3 S              33  39.8
4 T              25  30.1

Code lengths and letters in list# A tibble: 7 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           4 M               3   3.6
2           5 M              18  21.7
3           4 R               4   4.8
4           4 S              28  33.7
5           5 S               5   6  
6           4 T              23  27.7
7           5 T               2   2.4


In [26]:
single_nocms |>
  print(n=120)

# A tibble: 83 × 7
   hwa_code hwa_desc                                 code_length code_letter has_child is_child has_cms
   <chr>    <chr>                                          <int> <chr>       <lgl>     <lgl>    <lgl>  
 1 M074     Arthropathy in Crohn s disease K50 +               4 M           FALSE     FALSE    FALSE  
 2 M1199    Crystal arthropathy unsp site unsp                 5 M           FALSE     FALSE    FALSE  
 3 S497     Multiple injuries shoulder upper arm               4 S           FALSE     FALSE    FALSE  
 4 M1981    Other specified arthrosis shoulder                 5 M           FALSE     FALSE    FALSE  
 5 M1987    Other specified arthrosis ankle foot               5 M           FALSE     FALSE    FALSE  
 6 M1989    Other specified arthrosis site unsp                5 M           FALSE     FALSE    FALSE  
 7 M1999    Arthrosis unspecified site unspecified             5 M           FALSE     FALSE    FALSE  
 8 M2184    Oth spec acquired deformity limbs

It is posisble that these codes could be mapped using their chapter, i.e. by taking the code 1 level up and seeing if   
- Step 1) those codes are suitable (MSK)    
and    
- Step 2) whether they have 1-2-1 CMS maps   

Step 1:

In [27]:
single_nocms <- single_nocms |> 
  mutate(
    short_code = case_when(
      str_count(hwa_code) > 3 ~ str_sub(hwa_code, 1, -2),
      T ~ hwa_code
    )
  ) |> 
  left_join(
    who_2019,
    join_by(short_code == who_code)
  ) 

options(width=200)
single_nocms |>
  select(hwa_code,short_code, hwa_desc, who_desc) |>
  print(n=120)

# A tibble: 83 × 4
   hwa_code short_code hwa_desc                                 who_desc                                                                         
   <chr>    <chr>      <chr>                                    <chr>                                                                            
 1 M074     M07        Arthropathy in Crohn s disease K50 +     Psoriatic and enteropathic arthropathies                                         
 2 M1199    M119       Crystal arthropathy unsp site unsp       Crystal arthropathy, unspecified                                                 
 3 S497     S49        Multiple injuries shoulder upper arm     Other and unspecified injuries of shoulder and upper arm                         
 4 M1981    M198       Other specified arthrosis shoulder       Other specified arthrosis                                                        
 5 M1987    M198       Other specified arthrosis ankle foot     Other specified arthrosis                

THey all look fine. See how many have 1-2-1 mapping in the CMS list

In [28]:
single_nocms_map <- single_nocms |> 
  select(hwa_code, hwa_desc, short_code, code_length, code_letter) |>
  crossing(cms) |>
  filter(
   cms_code == short_code
  )
options(width=200)
single_nocms_map |>
  select(hwa_code, hwa_desc, cms_desc, short_code,cms_code) |>
  print(n=120)

# A tibble: 58 × 5
   hwa_code hwa_desc                                 cms_desc                                                                                                                        short_code cms_code
   <chr>    <chr>                                    <chr>                                                                                                                           <chr>      <chr>   
 1 M074     Arthropathy in Crohn s disease K50 +     Enteropathic arthropathies                                                                                                      M07        M07     
 2 M1199    Crystal arthropathy unsp site unsp       Crystal arthropathy, unspecified                                                                                                M119       M119    
 3 M1999    Arthrosis unspecified site unspecified   Osteoarthritis, unspecified site                                                                                            

Of these, the following codes will need to be removed:
Z094 - not an MSK FU   
T141, T143,T144, T146: the mapping is too vague in ICD-10-CM, T14 contains subcodes for suicide   

In [29]:
single_nocms_map <- single_nocms_map |>
  filter(
    !short_code %in% c("Z094", "T141", "T143", "T144", "T146")
  ) |> 
  select(hwa_code, hwa_desc, short_code, code_length, code_letter) |> 
  crossing(cms) |>
  filter(
    startsWith(cms_code, short_code)
  ) |> 
  select(-short_code) |> 
  # I dont need the codes I have already mapped using the parent  and single codes with 1-2-1 maps
  anti_join(
    cms_join,
    join_by(cms_code)
  ) 

describe_function(single_nocms_map, cms_code)
  

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1  5992

Code lengths in list# A tibble: 2 × 2
# Groups:   code_length [2]
  code_length     n
        <int> <int>
1           4  4879
2           5  1113

Code letters in list# A tibble: 4 × 3
# Groups:   code_letter [4]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 M            1454  24.3
2 R              38   0.6
3 S            4460  74.4
4 T              40   0.7

Code lengths and letters in list# A tibble: 6 × 4
  code_length code_letter     n   `%`
        <int> <chr>       <int> <dbl>
1           4 M            1049  17.5
2           5 M             405   6.8
3           4 R              38   0.6
4           4 S            3752  62.6
5           5 S             708  11.8
6           4 T              40   0.7


Create the final dataset. Use the allow combinations tribble to keep codes of the correct length for each code letter

In [30]:
cms_final <- bind_rows(cms_join, single_nocms_map) 

# Describe the final codes

In [31]:
describe_function(cms_final, cms_code)

[1] "N codes in list"
# A tibble: 1 × 1
      n
  <int>
1 26919

Code lengths in list# A tibble: 3 × 2
# Groups:   code_length [3]
  code_length     n
        <int> <int>
1           3   669
2           4 24868
3           5  1382

Code letters in list# A tibble: 8 × 3
# Groups:   code_letter [8]
  code_letter     n   `%`
  <chr>       <int> <dbl>
1 D               1   0  
2 G              32   0.1
3 I              27   0.1
4 M            2535   9.4
5 Q               1   0  
6 R              49   0.2
7 S           24230  90  
8 T              44   0.2

Code lengths and letters in list# A tibble: 13 × 4
   code_length code_letter     n   `%`
         <int> <chr>       <int> <dbl>
 1           4 D               1   0  
 2           4 G              32   0.1
 3           4 I              27   0.1
 4           3 M             162   0.6
 5           4 M            1898   7.1
 6           5 M             475   1.8
 7           4 Q               1   0  
 8           3 R               1   0  


In [39]:
hwa_figure_a <- hwa |> 
  mutate(
    letter = str_sub(hwa_code, 1, 1)
  )  |> 
  group_by(letter) |>
  mutate(
    letter = factor(letter, 
    # levels = c("Not Mapped","M","S","T","D", "G","I","Q","R")
    ),
    n = round(n()/nrow(hwa)*100,1)
  ) |>
  ungroup() |> 
  distinct(letter, n) |> 
  mutate(
    code = "hwa"
  )

hwa_figure_b <- hwa |>
  left_join(
    cms_final,
      join_by(hwa_code)
  ) |> 
  filter(
    !is.na(cms_code)
  ) |>
  mutate(
    letter = ifelse(is.na(cms_code), "Not Mapped", str_sub(cms_code, 1, 1))
  ) |>
  group_by(letter) |>
  mutate(
    n = round(n()/nrow(cms_final)*100,1)
  ) |>
  ungroup() |>
  distinct(letter, n) |>
  mutate(
    letter = factor(
      letter, 
      # levels = c("Not Mapped","M","S","T","D", "G","I","Q","R")
    ),
    code = "CMS"
  )

hwa_figure <- hwa_figure_a |>
  bind_rows(hwa_figure_b) |> 
  mutate(
    code = ifelse(code == "hwa", "ICD-10-AM", "ICD-10-CM"),
    code = factor(
      code, 
      levels = c("ICD-10-AM", "ICD-10-CM")
    )
  )

# Save

In [40]:
write.csv(cms_join, "hwa_joined.csv")
write.csv(hwa_figure, "hwa_figure.csv")